<a href="https://colab.research.google.com/github/yongbin02/P---AI/blob/main/Kmeans_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import joblib  # 모델 저장용
import json    # 매핑 파일 저장용

### 1. 경로 및 설정

In [5]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/Project_P/'
TRAIN_FILE = BASE_PATH + 'data/감성대화말뭉치(최종데이터)_Training.xlsx'
VAL_FILE = BASE_PATH + 'data/감성대화말뭉치(최종데이터)_Validation.xlsx'
NUM_CLUSTERS = 12 # 군집 개수

### 2. 데이터 로드 및 전처리

In [28]:
def load_and_preprocess(filepath):
    print(f"\n📂 파일 읽는 중... : {filepath}")
    try:
        df = pd.read_excel(filepath, engine='openpyxl')
    except Exception as e:
        print(f"❌ 파일 읽기 실패: {e}")
        return None

    # 필요한 컬럼 확인
    required_cols = ['사람문장1', '사람문장2', '사람문장3']
    target_cols = [col for col in required_cols if col in df.columns]

    if not target_cols:
        print(f"❌ 중요 컬럼이 없습니다.")
        return None

    # 수정 부분
    # Melt 대신 Apply 사용
    # NaN(빈칸)이 있을 수 있으므로 처리하며 병합
    def merge_sentences(row):
        texts = []
        for col in target_cols:
            sentence = str(row[col]).strip()
            # 'nan'이나 빈 문자열이 아니면 리스트에 추가
            if sentence and sentence.lower() != 'nan':
                texts.append(sentence)
        return " ".join(texts) # 띄어쓰기로 이어 붙임

    # 'text'라는 새로운 컬럼에 합친 문장
    df['text'] = df.apply(merge_sentences, axis=1)

    # 합쳤는데도 내용이 없으면 제거
    df_clean = df.dropna(subset=['text'])
    df_clean = df_clean[df_clean['text'].str.strip() != ""]

    # 필요한 컬럼만 남기고 리턴 (감정 라벨은 검증용으로 유지)
    return df_clean[['감정_대분류', 'text']]

In [29]:
print("--- 1. 데이터 로드 ---")
train_df = load_and_preprocess(TRAIN_FILE)
val_df = load_and_preprocess(VAL_FILE)

if train_df is None:
    print("❌ 데이터 로드 실패")
    exit()

print(f"✅ Training 데이터: {len(train_df)}개")
print(f"✅ Validation 데이터: {len(val_df)}개")

--- 1. 데이터 로드 ---

📂 파일 읽는 중... : /content/drive/MyDrive/Colab Notebooks/Project_P/data/감성대화말뭉치(최종데이터)_Training.xlsx

📂 파일 읽는 중... : /content/drive/MyDrive/Colab Notebooks/Project_P/data/감성대화말뭉치(최종데이터)_Validation.xlsx
✅ Training 데이터: 51630개
✅ Validation 데이터: 6641개


### 3. SBERT 임베딩 (벡터화)

In [ ]:
print("\n--- 2. SBERT 임베딩 시작 ---")
model = SentenceTransformer('jhgan/ko-sbert-multitask')
train_vectors = model.encode(train_df['text'].tolist(), show_progress_bar=True)

### 4. PCA 차원 축소

In [31]:
print(f"\n--- 3. PCA 차원 축소 시작 (768차원 -> 50차원) ---")
# n_components=50
# 이유: 12차원으로 줄였더니 정보 손실이 심함 -> 50차원에서도 효과적인 정보량 보존 및 노이즈 제거가 가능
pca_reducer = PCA(n_components=50, random_state=42)

# 학습 데이터 변환
train_pca = pca_reducer.fit_transform(train_vectors)
print(f"✅ PCA 변환 완료! 데이터 형태: {train_pca.shape}")


--- 3. PCA 차원 축소 시작 (768차원 -> 50차원) ---
✅ PCA 변환 완료! 데이터 형태: (51630, 50)


### 5. K-Means 군집화 (PCA 데이터)

In [32]:
print(f"\n--- 4. K-Means 군집화 시작 (K={NUM_CLUSTERS}) ---")
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init='auto')

# 50차원 데이터로 학습
kmeans.fit(train_pca)

# 데이터에 군집 번호 붙이기
train_df['cluster'] = kmeans.labels_


--- 4. K-Means 군집화 시작 (K=12) ---


### 6. 결과 분석

In [33]:
print("\n--- 5. 군집별 내용 분석 ---")
cluster_analysis = []

for i in range(NUM_CLUSTERS):
    cluster_data = train_df[train_df['cluster'] == i]
    if len(cluster_data) == 0: continue

    dominant_emotion = cluster_data['감정_대분류'].value_counts().idxmax()
    emotion_percent = (cluster_data['감정_대분류'].value_counts().max() / len(cluster_data)) * 100

    num_samples = min(3, len(cluster_data))
    examples = cluster_data['text'].sample(num_samples, random_state=42).values

    print(f"--- [Cluster {i}] 주된 감정: {dominant_emotion} ({emotion_percent:.1f}%) ---")
    for idx, ex in enumerate(examples):
        print(f"   예시 {idx+1}: {ex}")
    print("-" * 50)

    cluster_analysis.append({
        'cluster_id': i,
        'dominant_emotion': dominant_emotion,
        'purity': round(emotion_percent, 2),
        'examples': examples.tolist()
    })


--- 5. 군집별 내용 분석 ---
--- [Cluster 0] 주된 감정: 슬픔 (22.5%) ---
   예시 1: 나의 부양 문제를 두고 싸우는 자식들 소리를 듣게 됐어. 어떻게 될지 불안해. 싸우는 자식들한테 실망했어. 자식들한테 양로원에 들어간다고 얘기해야겠어.
   예시 2: 아이 낳고 본인 아이를 버리는 사람들을 보면 진짜 화가 나는 것 같아. 너무 생명을 쉽게 생각하는 것 같아. 그런 사람들은 벌을 크게 받아야 할 것 같아.
   예시 3: 이거 큰일이네. 급하게 천만 원 정도가 필요한데 빌릴 곳이 없네. 가족들에게 말하면 걱정할 거 같아서 아직 말 못 했어.
--------------------------------------------------
--- [Cluster 1] 주된 감정: 분노 (23.5%) ---
   예시 1: 예전부터 친하던 영순 할머니한테 무슨 일이 있는지 벌써 삼 년째 연락이 안 되서 불안해. 영순 할머니가 다른 동네로 이사를 간 뒤로는 연락이 끊겨서 무슨 일이 생겼나 걱정하고 있어. 일단 내일 경찰서에 가서 신고를 해야겠어.
   예시 2: 우리 가족은 대화가 없는거 같아. 대화가 없다 보니 가족 같지도 않고 서로 감정도 마비된 상태 같아.
   예시 3: 형이 혼자 방에 있을 때엔 들어가기 전에 꼭 노크를 해야 해서 조심스러워. 형은 노크해도 잘 못 들을 때마다 화내고 소리를 지르거든. 형에게 노크하면 좀 빨리 대답하라고 하고 싶어.
--------------------------------------------------
--- [Cluster 2] 주된 감정: 불안 (25.2%) ---
   예시 1: 남자 친구는 내 의견도 묻지 않고 결혼 후에 바로 아이를 갖자고 말해서 너무 당황스러워. 난 결혼 후에 바로 아이를 갖고 싶지 않은데 이 말을 할 때 남자 친구의 반응이 화가 나. 정확히 나의 의견을 말해서 둘이 같이 계획을 세워야겠어.
   예시 2: 아 오늘은 왜 이렇게 짜증이 나지? 오늘 생일인 친구가 있

### 7. Validation 검증 (파이프라인 적용)

In [34]:
print("\n--- 6. Validation 데이터 검증 ---")
# 1) SBERT 임베딩
val_vectors = model.encode(val_df['text'].tolist(), show_progress_bar=False)
# 2) PCA (Fit 하지 않고 Transform만)
val_pca = pca_reducer.transform(val_vectors)
# 3) K-Means 예측
val_df['predicted_cluster'] = kmeans.predict(val_pca)
print("✅ Validation 완료!")


--- 6. Validation 데이터 검증 ---
✅ Validation 완료!


### 8. 파일 저장

In [36]:
print("\n--- 7. 파일 저장 (구글 드라이브) ---")

# (1) 데이터 저장
train_df.to_excel(BASE_PATH + 'data/clustered_train_result_merged.xlsx', index=False)
print(f"✅ 분석 데이터 저장됨")

# (2) K-Means 모델 저장
joblib.dump(kmeans, BASE_PATH + 'data/kmeans_model.pkl')
print(f"✅ K-Means 모델 저장됨")

# (3) UMAP 모델 저장 -> kmeans와 함께 백엔드 서빙
joblib.dump(kmeans, BASE_PATH + 'data/pca_model.pkl')
print(f"✅ UMAP 모델 저장됨: pca_model.pkl (백엔드 서빙)")

# (4) 요약 정보
with open(BASE_PATH + 'cluster_summary_with_PCA.json', 'w', encoding='utf-8') as f:
    json.dump(cluster_analysis, f, ensure_ascii=False, indent=4)
print(f"✅ 요약 정보 저장됨")

print("\n🎉 저장 완료")


--- 7. 파일 저장 (구글 드라이브) ---
✅ 분석 데이터 저장됨
✅ K-Means 모델 저장됨
✅ UMAP 모델 저장됨: pca_model.pkl (백엔드 서빙)
✅ 요약 정보 저장됨

🎉 저장 완료


In [ ]:
# 파일은 kmeans_model.pkl과 pca_model.pkl 2개임
# 사용자 입력 -> SBERT -> pca_model.transform(벡터) -> kmeans_model.predict(압축된 벡터)
# 순서로 서빙해야 에러 안남